* set a context TF (D1,H4)
* Refresh all CDS from context to higher tf
* get a list of higher columns to merge


In [9]:
from jgtpy import JGTCDSSvc as svc

In [10]:
i="EUR/USD"
i="SPX500"
t="H4"
t="D1"

In [11]:
from jgtutils import jgtpov as jpov

In [12]:
povs=jpov.get_higher_tf_array(t)

In [13]:
use_fresh=False

In [14]:
use_full = True
quotescount=333
if use_full:
  quotescount=-1



In [15]:


for pov in povs:
  print(pov)

D1
W1
M1


In [16]:
#count the time this process takes
import time
force_read = True
start = time.time()
cdf_datasets=svc.get_higher_cdf_datasets(i,t,use_full=use_full,use_fresh=use_fresh,quotescount=quotescount,quiet=True,force_read=force_read)
#print how long it took
print("-------------------------------------")
print(time.time() - start)
print("-------------------------------------")

CDSSvc Read: CDSSvc Read: CDSSvc Read:    D1M1W1    of :  of :  of :    D1D1D1    for:   for:  for: SPX500   use_full:SPX500   True use_full:
SPX500  True use_full:
 True
-------------------------------------
0.2963709831237793
-------------------------------------


### B4 - ThreadPoolExecutor

* count:333, 4 timeframes = 22s
* count:FUL, 4 timeframes = 98s
* count:FUL, 4 timeframes = 80s  (With TP, not so much faster...), max_worker=default
* count:FUL, 4 timeframes = __s  (With TP, not so much faster...), max_worker=10


In [17]:
for k in cdf_datasets:
  
  #print(cdf)
  print(k)
  v=cdf_datasets[k]
  #print(v.columns)
  #print(v.tail(1))
  #print("")

M1
W1
D1


In [18]:
from jgtutils.jgtconstants import (MFI_VAL,ZCOL,AO)
columns_to_get_from_higher_tf = [MFI_VAL,ZCOL,AO]


In [19]:

workset=cdf_datasets.copy()
ttf=workset[t]
ttf.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-16 21:00:00,5429.92,5490.11,5421.07,5475.90,5430.32,5490.61,5421.60,5476.30,115164,5430.12,...,0,0,0,0,0,0,0,1,3,-+
2024-06-17 21:00:00,5475.90,5490.90,5470.58,5485.15,5476.30,5491.40,5471.18,5485.65,101284,5476.10,...,0,0,0,0,0,0,1,0,2,--


In [20]:
created_columns=[]
for c in columns_to_get_from_higher_tf:
  for k in workset:
    if not c in created_columns: 
      created_columns.append(c)
    new_col_name = c+"_"+k
    if k != t:
      if not new_col_name in created_columns: 
        created_columns.append(new_col_name)
print(created_columns)


['mfi_str', 'mfi_str_M1', 'mfi_str_W1', 'zcol', 'zcol_M1', 'zcol_W1', 'ao', 'ao_M1', 'ao_W1']


In [21]:

for k in workset:  
  if k!=t:
    v=workset[k]
    for c in columns_to_get_from_higher_tf:
     
      new_col_name = c+"_"+k
      ttf[new_col_name]=None

      for ii, row in ttf.iterrows():
        #get the date of the current row (the index)
        date = ii
        #print(k)
        data = v[v.index <= date]
        if not data.empty:
          data = data.iloc[-1]
          ttf.at[ii,new_col_name]=data[c]
        
      



In [22]:
print(created_columns)
#order the columns
#created_columns.sort()
#print(created_columns)


['mfi_str', 'mfi_str_M1', 'mfi_str_W1', 'zcol', 'zcol_M1', 'zcol_W1', 'ao', 'ao_M1', 'ao_W1']


In [28]:
#Print the ttf
#print(ttf.tail(5))

columns_we_want_to_keep_to_view="Volume,Open,High,Low,Close,ao,ac,jaw,teeth,lips,bjaw,bteeth,blips,tjaw,tteeth,tlips,fh,fl,fdbb,fdbs,fdb,zlc,zcol,mfi_str,M1_mfi_str,W1_mfi_str,M1_zcol,W1_zcol,W1_ao,M1_ao"
columns_we_want_to_keep_to_view="zcol,W1_zcol,M1_zcol,mfi_str,W1_mfi_str,M1_mfi_str,ao,W1_ao,M1_ao"
columns_we_want_to_keep_to_view=created_columns

#select columns in ttf
try:
  ttf_sel=ttf[columns_we_want_to_keep_to_view.split(",")].copy()
except:
  ttf_sel=ttf[columns_we_want_to_keep_to_view].copy()
  

ttf.to_csv("ttf.csv")
ttf_sel.to_csv("ttf_sel.csv")

In [24]:
# ttf=daily_df
# for ii, row in daily_df.iterrows():
#   #get the date of the current row (the index)
#   date = ii
#   #print(date)
#   for k in workset:  
#     if k!=t:
#       for c in columns_to_get_from_higher_tf:
#         new_col_name = k+"_"+c
#         ttf[new_col_name]=None
#         #print(k)
#         v=workset[k]
#         data = v[v.index <= date]
#         if not data.empty:
#           data = data.iloc[-1]
#           ttf.at[ii,new_col_name]=data[c]
        
      

In [25]:

ttf.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1'],
      dtype='object')

In [26]:
ttf.tail(1)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_fade,mfi_fake,mfi_sig,mfi_str,mfi_str_M1,zcol_M1,ao_M1,mfi_str_W1,zcol_W1,ao_W1
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-17 21:00:00,5475.9,5490.9,5470.58,5485.15,5476.3,5491.4,5471.18,5485.65,101284,5476.1,...,1,0,2,--,--,gray,0.747239,-+,gray,0.76117


* Iterations takes 1 min

## Above we are getting the columns from higher TF

In [27]:
    

# for k in workset:  
#   if k!=t:
#     print(k)
#     v=workset[k]
#     for c in columns_to_get_from_higher_tf:
#       new_col_name = k+"_"+c
#       print("Adding column for "+c+ " from tf:"+k + " as :"+new_col_name)
#       daily_df[new_col_name]=None
#       for ii, row in daily_df.iterrows():
#         date = ii
#         #print(date)
#         # Find the latest monthly data before or on the current date
#         v_data = v[v['Date'] <= date].iloc[-1]
#         daily_df.at[i, new_col_name] = v_data[c]
        
      
  
  